# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [2]:
!pip install langdetect

    100% |████████████████████████████████| 983kB 11.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for langdetect ... done
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [3]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from langdetect import detect

In [4]:
!ls

categories.csv			ETL Pipeline Preparation-zh.ipynb
disaster_messages.db		messages.csv
disaster_messages_v1.db		ML Pipeline Preparation.ipynb
disaster_messages_v2.db		ML Pipeline Preparation-zh.ipynb
DisasterResponse.db		Twitter-sentiment-self-drive-DFE.csv
ETL Pipeline Preparation.ipynb


In [5]:
# load messages dataset
messages = pd.read_csv('messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [6]:
# load categories dataset
categories = pd.read_csv('categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [7]:
# merge datasets
df = messages.merge(categories, on='id')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [8]:
# create a dataframe of the 36 individual category columns
categories = df.categories.str.split(';', expand=True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [9]:
list(categories.iloc[0,:].apply(lambda x: x[:-2]))

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [10]:
# select the first row of the categories dataframe
row = categories.iloc[0,:]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = list(row.apply(lambda x: x[:-2]))
print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [11]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [12]:
categories.related.apply(lambda x: x[-1]).astype(int).dtype

dtype('int64')

In [13]:
for column in categories:
    # set each value to be the last character of the string and then convert to 
    categories[column] = categories[column].apply(lambda x: x[-1]).astype(int)
 
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# check to see if all columns are binary
for col in categories:
    if categories[col].nunique() != 2:
        print(col)

related
child_alone


In [15]:
# investigating further
categories.related.value_counts()

1    20042
0     6140
2      204
Name: related, dtype: int64

In [16]:
categories.child_alone.value_counts()

0    26386
Name: child_alone, dtype: int64

In [17]:
# The related column has three values 0, 1 and 2 where as the child_alone
# column has a constant value of 0.
# Further investigation is required to understand what the three values 
# signify in the related column

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [18]:
# drop the original categories column from `df`
df.drop(['categories'], axis=1, inplace=True)

df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [19]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories], axis=1)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [20]:
df[~df.duplicated(keep='last')].shape

(26216, 40)

In [21]:
# check number of duplicates
df[df.duplicated(keep='last')].shape

(170, 40)

In [22]:
# drop duplicates
df = df[~df.duplicated(keep='last')]

In [23]:
# check number of duplicates
df[df.duplicated(keep='last')].shape

(0, 40)

In [24]:
# now looking into the related column further
df[df.related == 0][['message', 'original']].head(10)

,message,original
5,Information about the National Palace-,Informtion au nivaux palais nationl
8,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci
11,I am in Petionville. I need more information r...,M nan pv mvle plis enfomasyon sou 4636
17,are you going to call me or do you want me to ...,Eske se rele nap relem oubyen se mwen kap rele...
18,I don't understand how to use this thing 4636.,Mwen pa konprann koman pou m itilize bagay 463...
25,Can you tell me about this service,Can you tell me about this service
30,"Good evening, Radio one please. I would like i...",Bonswa radio one svp eskew pap k enfomasyon so...
32,"I'm here, I didn't find the person that I need...",Mwen la moun poum te voye pantalon mwen pa jwe...
52,I'm listening to you at Miraguan we asking the...,MWEN AP KOUTE NOU MIRAGWAN NOU AP MANDE POU RE...
66,"i am very happy, i hear god, religious hyme",Mwen tre kontan li mwen tande bondye fe ou la ...


In [25]:
df[df.related == 1][['message', 'original']].head(10)

,message,original
0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...
1,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini
2,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ..."
3,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...
4,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...
6,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus
7,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...
9,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ..."
10,"There's nothing to eat and water, we starving ...",Bon repo pa gen anyen menm grangou swaf
12,"I am in Thomassin number 32, in the area named...",Mwen thomassin 32 nan pyron mwen ta renmen jwe...


In [26]:
df[df.related == 2][['message', 'original']]

,message,original
117,Dans la zone de Saint Etienne la route de Jacm...,Nan zon st. etine rout jakmel la bloke se mize...
221,. .. i with limited means. Certain patients co...,t avec des moyens limites. Certains patients v...
307,The internet caf Net@le that's by the Dal road...,Cyber cafe net@le ki chita rout de dal tou pr ...
462,"Bonsoir, on est a bon repos aprs la compagnie ...",Bonswa nou nan bon repo apri teleko nan wout t...
578,URGENT CRECHE ORPHANAGE KAY TOUT TIMOUN CROIX ...,r et Salon Furterer. mwen se yon Cosmtologue. ...
660,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...
889,no authority has passed by to see us. We don't...,EN OKENN OTORITE POKO PASE WE NOU NOU PAGEN KO...
903,It's Over in Gressier. The population in the a...,Se over toujou nan gresye 24e seksyon komin. P...
931,we sleep with the baby. Thanks in advance for ...,sa nou demi avek ti bebe. Mesi davans pou d no...
937,I need help in Jrmie because I was in Port-au-...,MWEN bezwen ed lan jeremi paske mwen te portop...


In [57]:
# after observing that there are a few messages that are not in english,
# I will use the langdetect library to detect the language of each message
# and only keep ones which are english

from collections import defaultdict

langs = defaultdict(int)

for row in df.message:
    try:
        langs[detect(row)] += 1
    except:
        langs['could_not_detect'] += 1
print(langs)

defaultdict(<class 'int'>, {'en': 25808, 'no': 9, 'af': 33, 'nl': 25, 'cy': 7, 'fr': 55, 'ca': 19, 'de': 56, 'so': 24, 'tl': 9, 'et': 7, 'pt': 35, 'da': 17, 'fi': 2, 'hu': 1, 'it': 13, 'vi': 12, 'sl': 3, 'es': 18, 'sq': 6, 'sw': 5, 'could_not_detect': 5, 'hr': 3, 'ro': 2, 'pl': 2, 'tr': 4, 'id': 34, 'sv': 1, 'sk': 1})


In [27]:
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

In [28]:
df['is_english'] = df['message'].apply(is_english)

In [29]:
df.shape[0] - df[df.is_english].shape[0]

409

In [30]:
# As a result we will remove these 414 records

df = df[df.is_english].drop(['is_english'], axis = 1)

In [31]:
df.shape

(25807, 40)

In [33]:
df.related.value_counts()

1    19701
0     6037
2       69
Name: related, dtype: int64

In [37]:
# A decision has to be made regards to the three labels in the related column.
# As the number of records with label 2 is just 69, I will be dropping them.

df = df[df.related != 2]

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [73]:
engine = create_engine('sqlite:///disaster_messages_v2.db')
df.to_sql('disastermessages_v2', engine, index=False)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.